In [1]:
from flask import Flask

In [2]:
app = Flask("demo")
app.run(host = '0.0.0.0', port = 3040)

In [ ]:
# Simble predict
@app.route('/predict/<text>')
def predict(text):
    js = {}
    text = preprocess(text)
    
    vec = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(list(vec), maxlen)
    
    pred = model.predict(np.array(pad))[0]
    
    js['neg'] = float(pred[0])
    js['neu'] = float(pred[1])
    js['pos'] = float(pred[2])
    
    res = np.argmax(pred)
    
    if res == 2: 
        js['status'] = '<span style=\"background-color: #00FF00\">Positive</span>'
    elif res == 0:
        js['status'] = '<span style=\"background-color: #FF0000\">Negative</span>'
    else:
        js['status'] = '<span style=\"background-color: #FFFFFF\">Neutral</span>'
        
    
    return(json.dumps(js,ensure_ascii=False))

In [ ]:
# Predict (without one word) Attention

@app.route('/predict_gru/<text>')
def predict_GRU(text):
    #GRU
    js = {}
    
    time = datetime.datetime.now()
    
    text = preprocess(text)
    
    words = []
    for an in m.analyze(text):
        words.append(an['text'])
    words = words[:-1]
    lemmas = m.lemmatize(text)[:-1]
    
    for i,word in enumerate(lemmas):
        if word not in words_rare:
            lemmas[i] = 'rare'
            words[i] = 'rare'
            
    
    lemmas = ''.join(lemmas)
    vec = tokenizer.texts_to_sequences([lemmas])
    
    for i in range(0,len(vec[0])):
        vec.append(vec[0][:i] + vec[0][i+1:])
        
    pad = pad_sequences(list(vec), 100)
    pred = [1 - i[0] for i in model_CNN.predict(pad,batch_size = 128)]
    
    if pred[0] < threshold_GRU: 
        js['status'] = '<span style=\"background-color: #00FF00\">OK</span>'
    else:
        js['status'] = '<span style=\"background-color: #FF0000\">BAN</span>'
    
    time = str((datetime.datetime.now() - time).microseconds / 1000000) + ' s'
    
    k_good = (max(pred[1:]) - pred[0])/255
    k_bad = (min(pred[1:]) - pred[0])/255
    
    
    jswords = []
    for i in range(0, len(words)):
        jswords.append({
            "value": words[i],
            "color": '#FFFFFF'
        })
    
    for i in range(0,len(vec[0])):
        diff = pred[i + 1] - pred[0]
        for j in range(0,len(jswords)):
            if jswords[j]['value'] == words_map[vec[0][i]]:
                if (diff > 0):
                    coef = (pred[i + 1] - pred[0]) // k_good
                    if coef > 235:
                        coef = str('#0' + hex(255 - int(coef))[2:].upper() + 'FF' + '0' + hex(255 - int(coef))[2:].upper())
                        jswords[j]['color'] = coef
                    else:
                        coef = str('#' + hex(255 - int(coef))[2:].upper() + 'FF' +  hex(255 - int(coef))[2:].upper())
                        jswords[j]['color'] = coef
                else:
                    if (k_bad < - 0.00001):
                        coef = (pred[i + 1] - pred[0]) // k_bad
                        if coef > 235:
                            coef = str('#FF' + '0' + hex(255 - int(coef))[2:].upper() + '0' + hex(255 - int(coef))[2:].upper())
                            jswords[j]['color'] = coef
                        else:
                            coef = str('#FF' + hex(255 - int(coef))[2:].upper() + hex(255 - int(coef))[2:].upper())
                            jswords[j]['color'] = coef

    
    js['prediction'] = pred[0]
    js['time'] = time
    js['words'] = jswords
    js['threshold'] = threshold_GRU
    
    return(json.dumps(js,ensure_ascii=False))

In [ ]:
# Ridge Attention

@app.route('/predict_ridge/<text>')
def predict(text):
    #RIDGE
    js = {}
    
    time = datetime.datetime.now()
    
    text = preprocess(text)
    words = []
    for an in m.analyze(text):
        words.append(an['text'])
    words = words[:-1]
    lemmas = m.lemmatize(text)[:-1]
    
    for i,word in enumerate(lemmas):
        if word not in words_rare:
            lemmas[i] = 'rare'
            words[i] = 'rare'
        
    vec = vectorizer.transform([''.join(lemmas)])
    
    d = model_ridge.decision_function(vec)[0]
    pred = 1 - (np.exp(d) / (1 + np.exp(d)))
    js['prediction'] = float(pred)
    
    
    if pred < threshold_Ridge: 
        js['status'] = '<span style=\"background-color: #00FF00\">OK</span>'
    else:
        js['status'] = '<span style=\"background-color: #FF0000\">BAN</span>'
    
    time = str((datetime.datetime.now() - time).microseconds / 1000000) + ' s'
    
    
    jswords = []
    for i in range(0, len(lemmas)):
        word_id = vectorizer.vocabulary_.get(lemmas[i])
        if (word_id != None):
            coef = model.coef_[0][word_id] // k
            if coef > 0:
                if coef > 235: 
                    coef = str('#FF' + '0' + hex(255 - int(coef))[2:].upper() + '0' + hex(255 - int(coef))[2:].upper()) 
                else: 
                    coef = str('#FF' + hex(255 - int(coef))[2:].upper() + hex(255 - int(coef))[2:].upper())
                jswords.append({
                    "value": words[i],
                    "color": coef
                })
            else:
                jswords.append({
                    "value": words[i],
                    "color": '#FFFFFF'
                })
        else:
            jswords.append({
                "value": words[i],
                "color": '#FFFFFF'
            })
            
    js['time'] = time
    js['words'] = jswords
    js['threshold'] = threshold_Ridge
    
    return(json.dumps(js,ensure_ascii=False))